In [4]:
#import all the libraries
from modsim import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

**Stocks**: Uninfected Lymphocytes (R), Latently Infected Lymphocytes (L), Infected Lymphocytes (E), Virions (V)

**Parameters**: Uninfected Lymphocyte Production (Gamma), Lymphocyte Activation Rate (Tau), Lymphocyte natural death rate (Mu), Infection Rate Modifier (Beta), Proportion of cells latently infected (Rho), Latently Infected Activation Rate (Alpha), Infected Death Rate (Delta), Free Virion Production Rate (Pi), Free Virion Removal Rate (Sigma)

Gamma \* Tau is constant, Infection Rate = Beta * V

**Differential Equations**

dR/dt = Gamma\*Tau - Mu\*R - Beta\*R\*V

dL/dt = Rho\*Beta\*R\*V - Mu\*L - Alpha\*L

dE/dt = (1 - Rho)\*Beta\*R\*V + Alpha\*L - Delta\*E

dV/dt = Pi\*E - Sigma\*V

In [5]:
g = 0
t = 0
m = 0
b = 0
p = 0
a = 0
d = 0
p = 0
s = 0

0

In [6]:
def make_system(gamma, tau, mu, beta, rho, alpha, delta, pi, sigma):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(R = 0.0,
                 L = 0.0,
                 E = 0.0,
                 V = 0.0)

    t0 = 0
    t_end = 7 * 14

    return System(gamma = 0,
                  tau   = 0,
                  mu    = 0,
                  beta  = 0,
                  rho   = 0,
                  alpha = 0,
                  delta = 0,
                  pi    = 0, 
                  sigma = 0,
                  init  = init,
                  t0    = t0, 
                  t_end = t_end)


In [ ]:
def plot_results(R, L, E, V):
    plot(R, '--', label = 'Uninfected Lymphocytes')
    plot(L, '-',  label = 'Latently Infected Lymphocytes')
    plot(E, ':',  label = 'Infected Lymphocytes')
    plot(V, '-o', label = 'Free Virions')
    decorate(xlabel = 'Time (days)',
             ylabel = 'Fraction of population')

In [ ]:
def run_simulation(system, update_func):
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.row[t0] = init
    
    for t in linrange(t0, t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame

In [ ]:
def update_func(state, t, system):
    unpack(system)
    r, l, e, v = state

    dR = gamma*tau - mu*r - beta*r*v

    dL = rho*beta*r*v - mu*l - alpha*l

    dE = (1 - rho)*beta*r*v + alpha*l - delta*e

    dV = Pi*E - Sigma*V
    
    r += dR
    l += dL
    e += dE
    v += dV
    
    return State(R=r, L=l, E=e, V=v)

In [7]:
sys = make_system(g, t, m, b, p, a, d, p, s)

,values
gamma,0
tau,0
mu,0
beta,0
rho,0
alpha,0
delta,0
pi,0
sigma,0
init,R 0 L 0 E 0 V 0 dtype: int64
